# Useful Notebook: Report Testing Data Prediction Probabilities and Illustrate Accessing Evaluation Metrics
**This notebook will (1) show users how to access all model evaluation metrics from internal pickle files, and (2) generate model (class 1) prediction probabilities for instances of the respective testing dataset.**

*This notebook is designed to run after having run STREAMLINE (at least phases 1-6) and will use the files from a specific STREAMLINE experiment folder, as well as save new output files to that same folder.*

***
## Notebook Details
STREAMLINE outputs pickled objects with (1) all the metric results, (2) elements needed to build the ROC and PRC plots, as well as (3) the prediction probabilities on the testing data across all datasets, algorithm models, and CV dataset partitions. 

This notebook illustrates how the user can access the pickled metric information saved as a list object. 

It includes (1) grabbing and calculating all average metric scores over the CV partitions, (2) grabbing the elements needed to build the average ROC plot, (3) grabbing the elementes needed to build the average PRC plot, (4) grabbing and reporting average model feature importance scores, and (5) grabbing and reporting the model testing prediction probabilities for each instance of the dataset. 

When run, this last item will generate a new folder (`prediction_probas`) in the pipeline's output experiment folder in the `model_evaluation` folder for each dataset. Here the class 1 prediction probabilities are reported as a `.csv` file for each algorithm and CV partition pair. In these files is the instance's true outcome value, the unique instance ID, and the predicted probability of the instance being class 1 (i.e. which typically encodes cases or the less frequent class). 
 

***
## Notebook Run Parameters
* This notbook has been set up to run 'as-is' on the experiment folder generated when running the demo of STREAMLINE in any mode (if no run parameters were changed). 
* If you have run STREAMLINE on different target data or saved the experiment to some other folder outside of STREAMLINE, you need to edit `experiment_path` below to point to the respective experiment folder.

In [1]:
experiment_path = "../DemoOutput/demo_experiment" # path the target experiment folder 
target_data_list = None # None if user wants to generate output for all analyzed target datasets, otherwise provide a (str) list of target dataset names to run
algorithms = [] # use empty list if user wishes re-evaluate all modeling algorithms that were run in pipeline, otherwise specify a (str) list of algorithm identifiers.

***
## Housekeeping
### Import Packages

In [2]:
import os
import pandas as pd
import pickle
import numpy as np
from statistics import mean
from scipy import interp,stats
import warnings
warnings.filterwarnings('ignore')

# Jupyter Notebook Hack: This code ensures that the results of multiple commands within a given cell are all displayed, rather than just the last. 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Automatically Detect Dataset Names

In [3]:
# Get dataset paths for all completed dataset analyses in experiment folder
datasets = os.listdir(experiment_path)

# Name of experiment folder
experiment_name = experiment_path.split('/')[-1] 

datasets = os.listdir(experiment_path)
remove_list = ['.DS_Store', 'metadata.pickle', 'metadata.csv', 'algInfo.pickle',
                'DatasetComparisons', 'jobs', 'jobsCompleted', 'logs',
                'KeyFileCopy', 'dask_logs',
                experiment_name + '_ML_Pipeline_Report.pdf']
for text in remove_list:
    if text in datasets:
        datasets.remove(text)

datasets = sorted(datasets) # ensures consistent ordering of datasets
print("Analyzed Datasets: " + str(datasets))

Analyzed Datasets: ['hcc_data', 'hcc_data_custom']


### Load Other Necessary Parameters

In [4]:
# Unpickle metadata from previous phase
file = open(experiment_path + '/' + "metadata.pickle", 'rb')
metadata = pickle.load(file)
file.close()
# Load variables specified earlier in the pipeline from metadata
class_label = metadata['Class Label']
instance_label = metadata['Instance Label']
cv_partitions = int(metadata['CV Partitions'])

# Unpickle algorithm information from previous phase
file = open(experiment_path + '/' + "algInfo.pickle", 'rb')
algInfo = pickle.load(file)
file.close()
algorithms = []
abbrev = {}
for key in algInfo:
    if algInfo[key][0]: # If that algorithm was used
        algorithms.append(key)
        abbrev[key] = (algInfo[key][1])

print("Algorithms Ran: " + str(algorithms))

Algorithms Ran: ['Decision Tree', 'Logistic Regression', 'Naive Bayes']


***
## From Pickle: Extract Metric List and Cacluate CV Averages

In [5]:
def print_results(algorithm, full_path):
    # Define evaluation stats variable lists
        s_bac = [] # balanced accuracies
        s_ac = [] # standard accuracies
        s_f1 = [] # F1 scores
        s_re = [] # recall values
        s_sp = [] # specificities
        s_pr = [] # precision values
        s_tp = [] # true positives
        s_tn = [] # true negatives
        s_fp = [] # false positives
        s_fn = [] # false negatives
        s_npv = [] # negative predictive values
        s_lrp = [] # likelihood ratio positive values
        s_lrm = [] # likelihood ratio negative values
        
        aucs = [] #areas under ROC curve
        praucs = [] #area under PRC curve
        aveprecs = [] #average precisions for PRC
        
        for cv_count in range(0, cv_partitions): #loop through cv's
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            metric_list = results[0] #First item in pickled list is the metric list (set of standard classification metrics)
            roc_auc = results[3] #Fourth item is the ROC AUC
            prec_rec_auc = results[6] #Seventh item is the PRC AUC
            ave_prec = results[7] #Eighth item is the average precision of PRC
            
            #Separate metrics from metricList
            s_bac.append(metric_list[0])
            s_ac.append(metric_list[1])
            s_f1.append(metric_list[2])
            s_re.append(metric_list[3])
            s_sp.append(metric_list[4])
            s_pr.append(metric_list[5])
            s_tp.append(metric_list[6])
            s_tn.append(metric_list[7])
            s_fp.append(metric_list[8])
            s_fn.append(metric_list[9])
            s_npv.append(metric_list[10])
            s_lrp.append(metric_list[11])
            s_lrm.append(metric_list[12])
            
            aucs.append(roc_auc)
            praucs.append(prec_rec_auc)
            aveprecs.append(ave_prec)
            
        results = {'Balanced Accuracy': mean(s_bac), 'Accuracy': mean(s_ac), 
                   'F1_Score': mean(s_f1), 'Sensitivity (Recall)': mean(s_re), 
                   'Specificity': mean(s_sp),'Precision (PPV)': mean(s_pr), 
                   'TP': mean(s_tp), 'TN': mean(s_tn), 'FP': mean(s_fp), 
                   'FN': mean(s_fn), 'NPV': mean(s_npv), 'LR+': mean(s_lrp), 
                   'LR-': mean(s_lrm), 'ROC_AUC': mean(aucs),'PRC_AUC': mean(praucs), 
                   'PRC_APS': mean(aveprecs)}
        print(results)

In [6]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)
print("Datasets: " + str(datasets))
for each in datasets: 
    print("---------------------------------------")
    print(each)
    print("---------------------------------------")
    full_path = experiment_path + '/' + each
    for algorithm in algorithms: #loop through algorithms
        print(algorithm)
        print_results(algorithm, full_path)

Datasets: ['hcc_data', 'hcc_data_custom']
---------------------------------------
hcc_data
---------------------------------------
Decision Tree
{'Balanced Accuracy': 0.6206816059757236, 'Accuracy': 0.6060606060606061, 'F1_Score': 0.567090395480226, 'Sensitivity (Recall)': 0.6825396825396826, 'Specificity': 0.5588235294117647, 'Precision (PPV)': 0.4937282820578015, 'TP': 14, 'TN': 19, 'FP': 15, 'FN': 6, 'NPV': 0.7459733893557423, 'LR+': 1.6061980347694635, 'LR-': 0.5585525585525586, 'ROC_AUC': 0.6300186741363212, 'PRC_AUC': 0.5825474482082034, 'PRC_APS': 0.47488414874951873}
Logistic Regression
{'Balanced Accuracy': 0.6545284780578898, 'Accuracy': 0.6666666666666666, 'F1_Score': 0.5781653746770026, 'Sensitivity (Recall)': 0.6031746031746031, 'Specificity': 0.7058823529411765, 'Precision (PPV)': 0.5603993147852797, 'TP': 12, 'TN': 24, 'FP': 10, 'FN': 8, 'NPV': 0.744107744107744, 'LR+': 2.1252882919549583, 'LR-': 0.5626536426536427, 'ROC_AUC': 0.761204481792717, 'PRC_AUC': 0.690541771898

## From Pickle: Extract list of true and false positive rates for constructing ROC

In [7]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)
print("Datasets: " + str(datasets))

for each in datasets: 
    print("---------------------------------------")
    print(each)
    print("---------------------------------------")
    full_path = experiment_path+ '/' + each
    for algorithm in algorithms: #loop through algorithms
        print(algorithm)
        # Define evaluation stats variable lists
        tprs = [] # true postitive rates
        mean_fpr = np.linspace(0, 1, 100) # used to plot all CVs in single ROC plot
        
        for cv_count in range(0, cv_partitions): #loop through cv's
            # Load pickled metric file for given algorithm and cv =
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            fpr = results[1]
            tpr = results[2]

            tprs.append(interp(mean_fpr, fpr, tpr))
            tprs[-1][0] = 0.0

        results = {'tprs': np.mean(tprs, axis=0)}
        
        print(results)
        #print('fprs: '+str(mean_fpr))

Datasets: ['hcc_data', 'hcc_data_custom']
---------------------------------------
hcc_data
---------------------------------------
Decision Tree
{'tprs': array([0.        , 0.01713278, 0.03426556, 0.05139834, 0.06853112,
       0.0856639 , 0.10279667, 0.11992945, 0.13706223, 0.15419501,
       0.17132779, 0.18846057, 0.20559335, 0.22272613, 0.23985891,
       0.25699169, 0.27412446, 0.29125724, 0.30839002, 0.3255228 ,
       0.34265558, 0.35885843, 0.3744856 , 0.39011277, 0.40573994,
       0.42136711, 0.43699428, 0.45262145, 0.46824862, 0.48387579,
       0.5035113 , 0.52368126, 0.54385121, 0.56190476, 0.57989418,
       0.59721821, 0.6038961 , 0.61057399, 0.61725188, 0.62392977,
       0.63060766, 0.63728555, 0.64396344, 0.65064133, 0.65731922,
       0.66399711, 0.670675  , 0.67735289, 0.68403078, 0.69070867,
       0.69738656, 0.70406445, 0.71074234, 0.71742023, 0.72409812,
       0.73077601, 0.74114157, 0.7532708 , 0.76540003, 0.7752216 ,
       0.78433313, 0.79344465, 0.80119704,

## From Pickle: Extract list of precision and recall values for constructing PRC

In [8]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)
print("Datasets: " + str(datasets))

for each in datasets: 
    print("---------------------------------------")
    print(each)
    print("---------------------------------------")
    full_path = experiment_path + '/' + each
    for algorithm in algorithms: #loop through algorithms
        print(algorithm)
        # Define evaluation stats variable lists
        precs = [] # true postitive rates
        mean_recall = np.linspace(0, 1, 100) # used to plot all CVs in single PRC plot
        
        for cv_count in range(0, cv_partitions): #loop through cv's
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            prec = results[4]
            recall = results[5]

            precs.append(interp(mean_recall, recall, prec))

        results = {'precs': np.mean(precs, axis=0)}

        print(results)
        #print('recall: '+str(mean_recall))

Datasets: ['hcc_data', 'hcc_data_custom']
---------------------------------------
hcc_data
---------------------------------------
Decision Tree
{'precs': array([1.        , 0.98700339, 0.97400677, 0.96101016, 0.94801355,
       0.93501693, 0.92202032, 0.90902371, 0.89602709, 0.88303048,
       0.87003387, 0.85703725, 0.84404064, 0.83104403, 0.81804741,
       0.8050508 , 0.79205419, 0.77905757, 0.76606096, 0.75306435,
       0.74006773, 0.72707112, 0.71407451, 0.70107789, 0.68808128,
       0.67508467, 0.66208805, 0.64909144, 0.63609483, 0.62719905,
       0.61994361, 0.61268817, 0.60543273, 0.59817729, 0.59092184,
       0.5836664 , 0.57641096, 0.56915552, 0.56230047, 0.55644638,
       0.55059229, 0.54473821, 0.53888412, 0.53494735, 0.53244858,
       0.52994981, 0.52745103, 0.52495226, 0.52245348, 0.51995471,
       0.51745593, 0.51495716, 0.47559181, 0.47541349, 0.47523517,
       0.47505686, 0.47487854, 0.47470022, 0.4745219 , 0.47434358,
       0.47416526, 0.47398695, 0.47380863

## From Pickle: Extract Average Model Feature Importance Estimates (Over CVs)

In [9]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)
print("Datasets: " + str(datasets))

for each in datasets: 
    print("---------------------------------------")
    print(each)
    print("---------------------------------------")
    full_path = experiment_path + '/' + each
    original_headers = pd.read_csv(full_path + "/exploratory/ProcessedFeatureNames.csv", sep=',').columns.values.tolist() # Get Original Headers
    for algorithm in algorithms: #loop through algorithms
        print(algorithm)
        # Define evaluation stats variable lists
        FI_ave = [0] * len(original_headers)  # used to save average FI scores over all cvs. (all original features in dataset prior to feature selection included)
        
        for cv_count in range(0, cv_partitions): # loop through cv's
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Separate pickled results
            fi = results[8]
            
            # Format feature importance scores as list (takes into account that all features are not in each CV partition)
            tempList = []
            j = 0
            headers = pd.read_csv(full_path + '/CVDatasets/' + each + '_CV_' + str(cv_count) + '_Test.csv').columns.values.tolist()
            if instance_label != None: 
                headers.remove(instance_label)
            headers.remove(class_label)
            for feature in original_headers:
                if feature in headers:  # Check if current feature from original dataset was in the partition
                    # Deal with features not being in original order (find index of current feature list.index()
                    f_index = headers.index(feature)
                    FI_ave[j] += fi[f_index]
                j += 1
            
        #Turn FI sums into averages
        for i in range(0, len(FI_ave)):
            FI_ave[i] = FI_ave[i] / float(cv_partitions)

        fi_dict = {}
        for key in original_headers:
            for value in FI_ave:
                fi_dict[key] = value
                FI_ave.remove(value)
                break  
                
        print(fi_dict)

Datasets: ['hcc_data', 'hcc_data_custom']
---------------------------------------
hcc_data
---------------------------------------
Decision Tree
{'Gender': 0.0, 'Symptoms': 0.0, 'Alcohol': 0.0, 'Hepatitis B Surface Antigen': 0.0, 'Hepatitis B e Antigen': 0.0, 'Hepatitis B Core Antibody': 0.0, 'Hepatitis C Virus Antibody': 0.0, 'Cirrhosis': 0.0, 'Endemic Countries': 0.0, 'Smoking': 0.0, 'Diabetes': 0.0, 'Obesity': 0.0, 'Hemochromatosis': 0.0, 'Arterial Hypertension': 0.0, 'Chronic Renal Insufficiency': 0.0, 'Human Immunodeficiency Virus': 0.0, 'Nonalcoholic Steatohepatitis': 0.0, 'Esophageal Varices': 0.0, 'Splenomegaly': 0.0, 'Portal Hypertension': 0.0, 'Portal Vein Thrombosis': 0.0, 'Liver Metastasis': 0.0, 'Radiological Hallmark': 0.0, 'Age at diagnosis': 0.0, 'Grams of Alcohol per day': 0.00951213818860878, 'Packs of cigarets per year': 0.0, 'Performance Status*': 0.0584150326797386, 'Encephalopathy degree*': 0.0, 'Ascites degree*': 0.0, 'International Normalised Ratio*': 0.0, 'Alph

## Extract and Output Testing Data Prediction Probabilities 

In [10]:
if target_data_list: # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each in target_data_list:
            datasets.remove(each)
print("Datasets: " + str(datasets))

for each in datasets: 
    print("---------------------------------------")
    print(each)
    print("---------------------------------------")

    full_path = experiment_path + '/' + each
    
    # Make folder in experiment folder/datafolder to store all prediction probabilities per algorithm/CV combination
    if not os.path.exists(full_path + '/model_evaluation/prediction_probas'):
        os.mkdir(full_path + '/model_evaluation/prediction_probas')
        
    original_headers = pd.read_csv(full_path + "/exploratory/OriginalFeatureNames.csv", sep=',').columns.values.tolist() #Get Original Headers
    for algorithm in algorithms: #loop through algorithms
        print(algorithm)

        for cv_count in range(0,cv_partitions): #loop through cv's
            print(cv_count)
            #Load pickled metric file for given algorithm and cv
            result_file = full_path + '/model_evaluation/pickled_metrics/' + abbrev[algorithm] + "_CV_" + str(cv_count) + "_metrics.pickle"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            
            #Load associated testing dataset
            test_data = pd.read_csv(full_path + '/CVDatasets/'+each+'_CV_' + str(cv_count) + '_Test.csv')
            probas_summary = test_data[[class_label,instance_label]]

            #Separate pickled results
            probas_ = results[9]
            print(probas_[:,1])
            probas_summary['1_prob'] = probas_[:,1]
            file_name = full_path + '/model_evaluation/prediction_probas/' + algorithm + '_CV_'+str(cv_count) + '_class1_probas.csv'
            probas_summary.to_csv(file_name, index=False)

Datasets: ['hcc_data', 'hcc_data_custom']
---------------------------------------
hcc_data
---------------------------------------
Decision Tree
0
[0.95104895 0.95104895 0.30155211 0.95104895 0.95104895 0.95104895
 0.12830189 0.66019417 0.95104895 0.37777778 0.95104895 0.95104895
 0.66019417 0.39306358 0.95104895 0.82926829 0.39306358 0.39306358
 0.37777778 0.82926829 0.66019417 0.39306358 0.30155211 0.95104895
 0.12830189 0.30155211 0.82926829 0.30155211 0.30155211 0.
 0.12830189 0.66019417 0.12830189 0.39306358 0.82926829 0.12830189
 0.         0.95104895 0.66019417 0.37777778 0.37777778 0.
 0.37777778 0.39306358 0.30155211 0.30155211 0.39306358 0.30155211
 0.12830189 0.37777778 0.30155211 0.95104895 0.82926829 0.95104895
 0.12830189]
1
[0.51276102 0.11333333 0.11333333 0.51276102 0.80804388 0.51276102
 0.80804388 0.80804388 0.11333333 0.80804388 0.11333333 0.80804388
 0.11333333 0.80804388 0.51276102 0.51276102 0.11333333 0.11333333
 0.11333333 0.80804388 0.51276102 0.80804388 0.808